FP is programming w/ functions. Emphasizes functions.
A relation between a set of inputs & set of permissible 
outputs with the property that each input is realted to exactly 
one output
no side-effects 

what is a side effect: 
1. non-mathematical func
2. effect on state of the world
3. observable from outside
But we stil need side-effects to read, output to screen, communicate over network

In [2]:
%%javascript
Jupyter.utils.load_extensions('vim_binding/vim_binding')

<IPython.core.display.Javascript object>

Expression:
+ Replace imperative constructs w/ funcs
+ Produce higher level abstractions
+ Simpler code using fewer concepts (keep relevant things in your head, less moving parts)


Testing
+ Status quo: ad hoc mocking
+ mocks within mocks returning mocks
+ leads to tests coupled to implmentation

Performance
+ Multiprocessing
+ pure funcs are trivial to parallelize
+ PyPy STM

FP in the news
+ Apple launched Swift
+ Java w/ lambdas and closures
+ F#, MS Research, strongly functional

Higher Order Funcs
+ take funcs as args
+ returns funcs as results
+ eg. map, filter, sort, decorators

In [13]:
# built in constructs in Python: 1st class funcs w/ closures
# closures are a set of variables which get carried around with function as its passed around 
# ie...capturing the enviornment
def outer():
    x = 1
    def inner():
        return x
    return inner

f = outer()
assert f()== 1
print("Ok")

Ok


In [ ]:
# Python lacks 
# 1) strong built in library of pure funcs
# 2) efficient immutable data structs
# 3) cheap recursion

# imagine for loops weren't invented yet, let's look at some recursion examples
def mysum(nums):
    if nums == []:
        return 0
    else:
        return nums[0] + mysum(nums[1:])
    
print(mysum([1,2,3]))
def add_all(nums):
    if nums == []:
        return []
    else:
        return [nums[0] + 1] + and_all(nums[1:])
# in-efficient, list concat could cause stack overflow
print(add_all([1,2,3]))

In [36]:
# OOP programmer may write this to abstract addn method
class Mapper(object):
    def compute(self, element):
        raise NotImplementedError("Subclass and override me!")
    
    def map(self, l):
        if l == []:
            return l
        return [self.compute(l[0])] + self.map(l[1:])
    
# extract common functionality with FP, note how it differs from OOP
def mymap(f,l):
    if l == []:
        return l
    return [f(l[0])] + mymap(f, l[1:])

print(mymap(lambda n: n+2, [1,2,3]))

[3, 4, 5]


In [39]:
#consider the following 2 funcs
def mysum(nums):
    if nums == []:
        return 0
    else:
        return nums[0] + mysum(nums[1:])
mysum([1,2,3])

def length(l):
    if l == []:
        return 0
    else: 
        return 1 + length(l[1:])
length([1,2])

2

In [43]:
# we could abstract two methods above using
def map_and_add_nums(f,l):
    if l == []:
        return 0
    else:
        return f(l[0]) + map_and_add_nums(f, l[1:])
print("Sum", map_and_add_nums(lambda n: n, [1,2,3]))
print("Length", map_and_add_nums(lambda n: 1, [1,2,3]))
# but this only works for lists with numbers

Sum 6
Length 3


In [44]:
def biggest(nums, current_biggest=0):
    if nums == []:
        return current_biggest
    else:
        bigger = nums[0] if nums[0] > current_biggest else current_biggest
        return biggest(nums[1:], bigger)
print(biggest([1,2,5,3,-1]))

5


In [45]:
# higher the abstractions allows us to optimize without having to refactor other parts of our code
# HOF allow us to generalize patterns in our code by factoring out common recurring parts
# by parametrizing common parts of our code
def myreduce(f,l,acc):
    for el in l:
        acc = f(el, acc)
    return acc

def mymap(f,l):
    return myreduce(lambda el, acc: acc + [f(el)], l, [])
def add1_all(l):
    return mymap(lambda n: n + 1, l)

print("Add1", add1_all([1,2,3]))

Add1 [2, 3, 4]


In [49]:
from __future__ import print_function

def trace(name, x, f):
    print("<{}({})>".format(name,x), end='')
    result = f(x)
    print("<{}({})>".format(name,x,result), end='')
    return result 

# this HOF makes the funcs below clear since it removes code which would be in each function 
def trace_decorator(f):
    return lambda x: trace(f.__name__, x, f)

# A function that takes another function as an argument, generates a new function, 
# augmenting the work of the original function, and returning the generated function so we 
# can use it anywhere. 
@trace_decorator
def do_work(x):
    return sub_work(x) + other_work(x)

@trace_decorator
def sub_work(x):
    return x * 2

@trace_decorator
def other_work(x):
    return x - 5
# instead of 
# def other_work(x):
# return trace("other_work", x, lambda x: x - 5)

do_work(50)
    

<do_work(50)><sub_work(50)><sub_work(50)><other_work(50)><other_work(50)><do_work(50)>

145

In [5]:
# if you have a list of list and want to mutate the last element, 
# we would copy the whole list into a new list and then mutate the last elem
# doubles necessary memory, takes lots of time if list is BIG
def update_last(l,v):
    new_l = l[:]
    new_l[-1] = v
    return new_l
l = [1,2,3]
update_last(l,-1)

[1, 2, -1]

Pyrsistent Types:
- pvector: lists
- pmap: dicts
- pset: sets
- pclass: classes
- plist: linked lists

How does Pyrsistent work?
- array mapped trie
- elements are stored in leaves not branches
- modifying a structure leaves original intact

Understanding Clojure's PersistentVector implementation
http://blog.higher-order.net/2009/02/01/understanding-clojures-persistentvector-implementation.html

In [11]:
import timeit
from pyrsistent import pvector

def bm_copy_and_append(size):
    l = list(range(size))
    def run():
        new_l = l[:]
        new_l.append('foobar')
    return timeit.timeit(run)
    
def bm_pyrsistent_append(size):
    vec = pvector(range(size))
    def run():
        return vec.append('foobar')
    return timeit.timeit(run)

bm_copy_and_append(1000)

3.4593809220004914

In [12]:
#note which is more performent 
bm_pyrsistent_append(1000)

0.33820987500075717

In [13]:
# immutable vs direct mutation, immutable is 2x slower but that's not a horrible tradeoff
import timeit
from pyrsistent import pvector

def bm_append(size):
    l = list(range(size))
    def run():
        l.append('foobar')
    return timeit.timeit(run)
    
def bm_pyrsistent_grow(size):
    vec = [pvector(range(size))]
    def run():
        vec[0] = vec[0].append('foobar')
    return timeit.timeit(run) 

bm_append(1000)

0.20225019399913435

In [14]:
bm_pyrsistent_grow(1000)

0.43875211799968383

Toolz: Functional Standard Library
1. toolz.functoolz
  1. compose
  2. curry ex: plus(1)(2)==3
2. toolz.itertoolz
  1. toolz.dicttoolz

3. functoolz extends stdlib functools
4. itertoolz extends stdlib itertools

In [4]:
# compose function plain python
def g(x):
    return x + 10
def f(x):
    return x * 5
def compose(x):
    return f(g(x))

result = []
for i in inputs:
    x = f(g(i))
    result.append(x)
print(result)
compose(3)

[55, 60, 65]


65

In [16]:
# compose using toolz
from toolz.functoolz import compose
compose(f,g)(3)

65

In [17]:
f(g(3))

65

In [21]:
# use compose when we need to pass compose to HOF
map(compose(f,g), [1,2,3])
# better than this
map(lambda x: f(g(x)), [1,2])

In [22]:
# example of currying
# in haskell, funcs can only take a single parameter, but multi-arg funcs are possible if 
# a func returns another func when called with each arg in turn.
# if fewer than necessary are passed, returns a partially applied func
from toolz.functoolz import curry
@curry
def plus(x,y):
    return x + y

plus(1,2)

3

In [23]:
plus(1)

<function plus at 0x1042b7c80>

In [24]:
plus(1)(2)

3

In [27]:
map(plus(1), [1,2,3])

In [28]:
from functools import partial
map(partial(plus,1), [1,2,3])

In [30]:
from toolz.functoolz import identity, memoize

identity(3)

@memoize 
def foo(x):
    return x + 3
foo(3)

6

In [31]:
i = iter([1,2,3])
i[2] # won't work

next(i);next(i);next(i)


TypeError: 'list_iterator' object is not subscriptable

In [41]:
from toolz.itertoolz import nth, last, drop, take, groupby, interpose, first
nth(2, iter([1,2,3]))
last(iter([1,2,3]))

3

In [36]:
i = iter([1,2,3])
last(i)
try: last(i)
# python iterators are impure, consuming them mutates their internal state. Can't call last func 
# twice since it consumes the whole thing

SyntaxError: unexpected EOF while parsing (<ipython-input-36-9ff66c40ea4e>, line 5)

In [39]:
list(drop(2,iter([1,2,3])))
list(take(2,iter([1,2,3])))

[1, 2]

In [42]:
groupby(first,['ABC','ABA','BAB','BAA'])

{'A': ['ABC', 'ABA'], 'B': ['BAB', 'BAA']}

In [44]:
# takes iterable returns iterator where 1st argument is inserted between each elem in given iterable
list(interpose('foo', [1,2,3]))

[1, 'foo', 2, 'foo', 3]

In [55]:
# image if you had to merge 2 dicts
d1 = {'foo':'bar'}
d2 = {'baz': 'quux'}
new_d = d1.copy()
new_d.update(d2)
new_d

{'baz': 'quux', 'foo': 'bar'}

In [48]:
from toolz.dicttoolz import merge, assoc, dissoc, get_in
merge(d1,d2)

{'baz': 'quux', 'foo': 'bar'}

In [51]:
assoc(d1,'a',1)

{'a': 1, 'foo': 'bar'}

In [52]:
dissoc(d1, 'foo')

{}

In [56]:
help('functools')

Help on module functools:

NAME
    functools - functools.py - Tools for working with functions and callable objects

MODULE REFERENCE
    http://docs.python.org/3.5/library/functools
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

CLASSES
    builtins.object
        partial
        partialmethod
    
    class partial(builtins.object)
     |  partial(func, *args, **keywords) - new function with partial application
     |  of the given arguments and keywords.
     |  
     |  Methods defined here:
     |  
     |  __call__(self, /, *args, **kwargs)
     |      Call self as a function.
     |  
     |  __delattr__(self, name, /)
     |      Implement delattr(self, name).
     |  
     |  __getattribute__(self, nam

In [63]:
# Hypothesis: Describe inputs & formulate properties
def add2(num):
    return num + 2

def test_adds_2():
    assert add2(0) == 2
    assert add2(5) == 7 
    assert add2(-5) == -3

test_adds_2()

In [66]:
from hypothesis import given
from hypothesis import strategies as st

def test_adds_2(num):
    assert add2(num) == num + 2
# generates by default 200 randon ints and runs it on our func
test_adds_2

<function __main__.test_adds_2>

In [68]:
# sample strategy
from operator import add
def mysum(nums):
    return reduce(add, nums, 0)
def test_mysum_plain():
    assert mysum([1,2,3]) == 6
test_mysum_plain()

NameError: name 'reduce' is not defined

In [69]:
@given(st.lists(st.integers()))
def test_mysum(nums):
    assert mysum(nums) == sum(nums)
test_mysum()

Falsifying example: test_mysum(nums=[])


NameError: name 'reduce' is not defined

Object `reduce` not found.
